In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy
import moose

In [2]:
def displayPlots():
    '''Plotting concentration of reacting entities over time.'''
    for x in moose.wildcardFind( '/model/graphs/conc#/#' ):
        t = numpy.arange( 0, x.vector.size, 1 ) #sec
        plt.plot( t, x.vector, label=x.name )
        plt.xlabel("Time(s)")
        plt.ylabel("Concentration (mM)")
    plt.legend(bbox_to_anchor=(0., 1.02, 1., .102), loc=3,
           ncol=3, mode="expand", borderaxespad=0.)
    plt.show()

In [3]:
def getState( state ):
    '''Randomize initial conditions and find steady states'''
    state.randomInit()
    moose.start( 2 ) # Run the model for 2 seconds.
    state.settle()        
    moose.start( 100.0 ) # Run model for 10 seconds, just for display
    steadyStateVector = [  y.conc for y in moose.wildcardFind( '/model/kinetics/##[ISA=PoolBase]')]
    return steadyStateVector

In [4]:
moose.loadModel( 'M1719.g', '/model', 'ee' )

<moose.vec: class=Neutral, id=458, path=/model>

![Model M1719](strongBis.png "Bistable model")

In [ ]:
compartment = moose.element( 'model/kinetics' )
ksolve = moose.Ksolve( '/model/kinetics/ksolve' )

stoich = moose.Stoich( '/model/kinetics/stoich' )
stoich.compartment = compartment
stoich.ksolve = ksolve
stoich.path = "/model/kinetics/##"

state = moose.SteadyState( '/model/kinetics/state' )

moose.reinit()
state.stoich = stoich

In [ ]:
for i in range( 0, 5 ):
    SS_vector = getState( state )
    #print SS_vector

moose.start( 100.0 ) # Run the model for 100 seconds.

b = moose.element( '/model/kinetics/b' )
c = moose.element( '/model/kinetics/c' )

# move most molecules over to b
b.conc = b.conc + c.conc * 0.95
c.conc = c.conc * 0.05
moose.start( 100.0 ) # Run the model for 100 seconds.

# move most molecules back to a
c.conc = c.conc + b.conc * 0.95
b.conc = b.conc * 0.05
moose.start( 100.0 ) # Run the model for 100 seconds.

# Iterate through all plots, dump their contents to data.plot.
displayPlots()